In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import community as community_louvain



def get_suit_number(num):
    try:
        num = float(num)
        return num
    except:
        return 0
    
    
    
for process_year in range(2010,2021):
    # 获得这年得年报
    data = pd.read_excel(r"{}年报.xlsx".format(process_year), dtype={'代码': str})
    # 排除掉一般法人和非金融上市公司
    select_data = data.loc[np.array(data["机构类型"] != '一般法人') & np.array(data["机构类型"] != '非金融类上市公司')]
    institution_names = list(set(select_data["机构名称"]))
    print("一共有{}家机构".format(len(institution_names)))
    matrix = np.zeros([len(institution_names),len(institution_names)])
    company_names = list(set(select_data["上市公司名称"]))
    print("一共有{}家上市公司".format(len(company_names)))

    
    percent = 5
    for timer,cpname in enumerate(company_names):
        if timer%200 == 0:
            print("完成{0}/{1}".format(timer,len(company_names)))
        cp_data = select_data.loc[select_data["上市公司名称"] == cpname]
        cp_inst_names = cp_data["机构名称"].to_list()
        cp_a_share = cp_data["占流通A股比例(%)"].apply(get_suit_number).to_list()
        if cp_a_share[0] >= percent/2:
            for i in range(len(cp_inst_names) - 1):
                for j in range(i + 1, len(cp_inst_names)):
                    if cp_a_share[i] + cp_a_share[j] >= percent:
                        matrix[institution_names.index(cp_inst_names[i]), institution_names.index(cp_inst_names[j])] = 1

    net = nx.Graph(matrix)
    partion = community_louvain.best_partition(net, resolution=1.0, random_state=0)
    cmt = list(partion.values())
    result = pd.DataFrame({"机构名称": institution_names, "团体编号": cmt})
    result.to_excel("{}年louvian结果.xlsx".format(process_year), index = False)
    
    
    count = {}
    for i in cmt:
        count[i] = count.get(i,0) + 1
    sort_count = sorted(count.items(), key = lambda x:x[1], reverse=True)

    community_ids = []
    community_member_num = []
    for (key,val) in sort_count:
        if val >= 2:
            community_ids.append(key)
            community_member_num.append(val)
    print("一共存在{}个团体".format(len(community_ids)))
    timer = 0 
    new_id = 1
    for community_id in community_ids:
        for name in np.array(institution_names)[np.array(cmt) == community_id]:
            community_data = select_data.loc[select_data["机构名称"] == name]
            community_data.insert(0,"团体编号",new_id)
            if timer == 0:
                update_data = community_data
                timer += 1
            else:
                update_data = update_data.append(community_data, ignore_index=True)
        new_id += 1

    update_data.to_excel("{}年团体结果.xlsx".format(process_year), index = False)

一共有568家机构
一共有1547家上市公司
完成0/1547
完成200/1547
完成400/1547
完成600/1547
完成800/1547
完成1000/1547
完成1200/1547
完成1400/1547
一共存在11个团体
一共有615家机构
一共有1881家上市公司
完成0/1881
完成200/1881
完成400/1881
完成600/1881
完成800/1881
完成1000/1881
完成1200/1881
完成1400/1881
完成1600/1881
完成1800/1881
一共存在10个团体
一共有615家机构
一共有1932家上市公司
完成0/1932
完成200/1932
完成400/1932
完成600/1932
完成800/1932
完成1000/1932
完成1200/1932
完成1400/1932
完成1600/1932
完成1800/1932
一共存在9个团体
一共有737家机构
一共有2006家上市公司
完成0/2006
完成200/2006
完成400/2006
完成600/2006
完成800/2006
完成1000/2006
完成1200/2006
完成1400/2006
完成1600/2006
完成1800/2006
完成2000/2006
一共存在17个团体
一共有957家机构
一共有2189家上市公司
完成0/2189
完成200/2189
完成400/2189
完成600/2189
完成800/2189
完成1000/2189
完成1200/2189
完成1400/2189
完成1600/2189
完成1800/2189
完成2000/2189
一共存在14个团体
一共有1044家机构
一共有2495家上市公司
完成0/2495
完成200/2495
完成400/2495
完成600/2495
完成800/2495
完成1000/2495
完成1200/2495
完成1400/2495
完成1600/2495
完成1800/2495
完成2000/2495
完成2200/2495
完成2400/2495
一共存在15个团体
一共有1297家机构
一共有2707家上市公司
完成0/2707
完成200/2707
完成400/2707
完成600/2707
完成800/2707
完成1000/2707

# 获得透视结果啥的

In [42]:
for process_year in range(2010,2021):
    # 获得这年得年报
    data = pd.read_excel(r"{}年报.xlsx".format(process_year), dtype={'代码': str})
    # 排除掉一般法人和非金融上市公司
    select_data = data.loc[np.array(data["机构类型"] != '一般法人') & np.array(data["机构类型"] != '非金融类上市公司')]
    codes = list(set(select_data["代码"]))
    timer = 1
    for code in codes:
        code_data = data.loc[data["代码"] == code]
        try:
            occupy = np.array(code_data["占流通A股比例(%)"].astype("float"))
        except:
            continue
        # 四个指标 合计、最大值、最小值和平方和
        sum_val = occupy.sum()
        min_val = occupy.min()
        max_val =occupy.max()
        power_sum_val = np.power(occupy, 2).sum()
        columns = ["公司名称","公司代码","和", "最大值", "最小值", "平方和"]
        # 修改columns
        output = pd.DataFrame([[code_data.iloc[0,0], code, sum_val, max_val, min_val, power_sum_val]], columns=columns)
        if timer == 1:
            new_output = output
            timer = -1
        else:
            new_output = new_output.append(output, ignore_index=True)
    new_output.to_excel("{}年各公司指标.xlsx".format(process_year), index = False)